In [1]:
%load_ext autoreload

In [3]:
import os
os.chdir("..")

In [4]:
%autoreload 2
import pandas as pd
import numpy as np
%aimport AD_comparison_tools
%aimport AD_predictor_tools
%aimport ADpred_LambertTFs_helper
import protfasta

In [28]:
yeast_TFs = pd.read_csv("../output/yeast_TF_seqs.csv", index_col = 0)
yeast_TFs = yeast_TFs.rename(columns = {"seq" : "sequence"})
yeast_TFs["uniprotID"] = yeast_TFs["id"].str.split("|").str[1]
yeast_TFs = yeast_TFs.drop(columns = "id")
yeast_TFs

,sequence,uniprotID
0,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,O14467
1,MENTILHIHSFQLPQTEQPYPEAMLFDRDTSDSRTVLTQKPNGLEI...,O93958
2,MSEYQPSLFALNPMGFSPLDGSKSTNENVSASTSTAKPMVGQLIFD...,P03069
3,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...,P04386
4,MDYNKRSSVSTVPNAAPIRVGFVGLNAAKGWAIKTHYPAILQLSSQ...,P04387
...,...,...
242,MPSRTPTDIWRCQRLILAARKGETTCQALHEQSIEISSSLKWFEEI...,Q707Y3
243,MQFTILNEPSLDSQRREGDLASENYVFGDIRKEGVRILEDSLRSER...,Q707Y6
244,MNVQEIHNIREACITILSGTKHNSVLFEPCDKFDEVINSLDIDPDS...,Q708A1
245,MCDNDMADIQSKLSSFCEEIRALALKEGYNLEGDKSPSSKPYFMSW...,Q9HG12


In [40]:
preds = pd.read_csv("../output/adpred_yeast/yeast_TF/ADpred_preds.csv", index_col = 0)
preds

,sequence,adpred_preds
0,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,"[0.003913919907063246, 0.003878817893564701, 0..."
1,MENTILHIHSFQLPQTEQPYPEAMLFDRDTSDSRTVLTQKPNGLEI...,"[0.006153729744255543, 0.0037436150014400482, ..."
2,MSEYQPSLFALNPMGFSPLDGSKSTNENVSASTSTAKPMVGQLIFD...,"[0.021538162603974342, 0.06201121211051941, 0...."
3,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...,"[0.004221274517476559, 0.0089436499401927, 0.0..."
4,MDYNKRSSVSTVPNAAPIRVGFVGLNAAKGWAIKTHYPAILQLSSQ...,"[0.00021456548711284995, 0.0002215386484749615..."
...,...,...
242,MPSRTPTDIWRCQRLILAARKGETTCQALHEQSIEISSSLKWFEEI...,"[0.0014189841458573937, 0.0035259572323411703,..."
243,MQFTILNEPSLDSQRREGDLASENYVFGDIRKEGVRILEDSLRSER...,"[0.0031145745888352394, 0.0017415679758414626,..."
244,MNVQEIHNIREACITILSGTKHNSVLFEPCDKFDEVINSLDIDPDS...,"[0.0011840685037896037, 0.002713506342843175, ..."
245,MCDNDMADIQSKLSSFCEEIRALALKEGYNLEGDKSPSSKPYFMSW...,"[0.005300438962876797, 0.02574380487203598, 0...."


In [30]:
preds = pd.merge(yeast_TFs, preds, on = "sequence").drop_duplicates().reset_index(drop = True)
preds

,sequence,uniprotID,adpred_preds
0,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,O14467,"[0.003913919907063246, 0.003878817893564701, 0..."
1,MENTILHIHSFQLPQTEQPYPEAMLFDRDTSDSRTVLTQKPNGLEI...,O93958,"[0.006153729744255543, 0.0037436150014400482, ..."
2,MSEYQPSLFALNPMGFSPLDGSKSTNENVSASTSTAKPMVGQLIFD...,P03069,"[0.021538162603974342, 0.06201121211051941, 0...."
3,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...,P04386,"[0.004221274517476559, 0.0089436499401927, 0.0..."
4,MDYNKRSSVSTVPNAAPIRVGFVGLNAAKGWAIKTHYPAILQLSSQ...,P04387,"[0.00021456548711284995, 0.0002215386484749615..."
...,...,...,...
242,MPSRTPTDIWRCQRLILAARKGETTCQALHEQSIEISSSLKWFEEI...,Q707Y3,"[0.0014189841458573937, 0.0035259572323411703,..."
243,MQFTILNEPSLDSQRREGDLASENYVFGDIRKEGVRILEDSLRSER...,Q707Y6,"[0.0031145745888352394, 0.0017415679758414626,..."
244,MNVQEIHNIREACITILSGTKHNSVLFEPCDKFDEVINSLDIDPDS...,Q708A1,"[0.0011840685037896037, 0.002713506342843175, ..."
245,MCDNDMADIQSKLSSFCEEIRALALKEGYNLEGDKSPSSKPYFMSW...,Q9HG12,"[0.005300438962876797, 0.02574380487203598, 0...."


In [43]:
def activity_string_to_array(prob_string):
    a = prob_string.replace('\n','')
    a = a.replace('[','')
    a = a.replace(']','').split(",")
    b = []
    for x in a:
        if x != "":
            b.append(float(x))
    return b

float_preds = []

for str_list in preds["adpred_preds"]:
    float_preds.append(activity_string_to_array(str_list))
    
preds["adpred_preds"] = float_preds
preds

,sequence,adpred_preds
0,MSDWDTNTIIGSRARAGGSGPRANVARSQGQINAARRQGLVVSVDK...,"[0.003913919907063246, 0.003878817893564701, 0..."
1,MENTILHIHSFQLPQTEQPYPEAMLFDRDTSDSRTVLTQKPNGLEI...,"[0.006153729744255543, 0.0037436150014400482, ..."
2,MSEYQPSLFALNPMGFSPLDGSKSTNENVSASTSTAKPMVGQLIFD...,"[0.021538162603974342, 0.06201121211051941, 0...."
3,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...,"[0.004221274517476559, 0.0089436499401927, 0.0..."
4,MDYNKRSSVSTVPNAAPIRVGFVGLNAAKGWAIKTHYPAILQLSSQ...,"[0.00021456548711284995, 0.0002215386484749615..."
...,...,...
242,MPSRTPTDIWRCQRLILAARKGETTCQALHEQSIEISSSLKWFEEI...,"[0.0014189841458573937, 0.0035259572323411703,..."
243,MQFTILNEPSLDSQRREGDLASENYVFGDIRKEGVRILEDSLRSER...,"[0.0031145745888352394, 0.0017415679758414626,..."
244,MNVQEIHNIREACITILSGTKHNSVLFEPCDKFDEVINSLDIDPDS...,"[0.0011840685037896037, 0.002713506342843175, ..."
245,MCDNDMADIQSKLSSFCEEIRALALKEGYNLEGDKSPSSKPYFMSW...,"[0.005300438962876797, 0.02574380487203598, 0...."


In [32]:
df_list = []

for i in range(len(preds.index)):
    df_list.append(ADpred_LambertTFs_helper.return_pred_df(preds["adpred_preds"][i], 
                                  0.8, 
                                  10,
                                  preds["sequence"][i],
                                  preds["uniprotID"][i]))

In [39]:
final_preds = pd.concat(df_list).drop(columns = "ProteinRegionSeq").reset_index(drop = True)
final_preds["Start"] = final_preds["Start"].astype(int)
final_preds["End"]  = final_preds["End"].astype(int)
final_preds

,GeneName,Start,End
0,O93958,211,225
1,O93958,256,273
2,P03069,109,124
3,P04386,167,177
4,P04386,853,863
...,...,...,...
316,Q12340,360,374
317,Q12340,542,559
318,Q12340,752,780
319,Q707Y3,34,59


In [45]:
final_preds.to_csv("../output/yeast_TFs_preds/cleaned/adpred.csv")